### Новове обучение

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
raw_model = 'ai-forever/ruT5-large'
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda();
tokenizer = T5Tokenizer.from_pretrained(raw_model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# file = '/content/drive/MyDrive/Аспирантура/dataset.csv'
file = '/content/dataset.csv'

In [ ]:
df = pd.read_csv(file).head(3000) # na_values=['NaN', 'nan']

In [ ]:
print(len(df))

586


In [ ]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.2, random_state=1)
pairs = df_train[['X', 'Y']].values.tolist()

In [ ]:
batch_size = 2  # сколько примеров показывем модели за один шаг 4 8
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 10  # сколько раз мы покажем данные модели 3 6

In [ ]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))


EPOCH 0


  0%|          | 0/234 [00:00<?, ?it/s]

step 0 loss 5.1073126792907715
step 200 loss 2.8360729694366453
EPOCH 1


  0%|          | 0/234 [00:00<?, ?it/s]

step 0 loss 2.5661706846952437
step 200 loss 1.9792366316914558
EPOCH 2


...

step 0 loss 1.2070826755464077
step 200 loss 1.1794294704496862
EPOCH 8


  0%|          | 0/234 [00:00<?, ?it/s]

step 0 loss 1.1571262546628713
step 200 loss 1.0899290663003922
EPOCH 9


  0%|          | 0/234 [00:00<?, ?it/s]

step 0 loss 1.1041742861270905
step 200 loss 0.9766598964482546


In [ ]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
sample = df_train.sample(5)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', answer(row.X))
    print('---')

Термическая деструкция — это процесс разрушения макромолекул под влиянием повышенных температур.
real: разрушение макромолекул; Температура;


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


model:  Температура;
---
Формируют нечеткую базу знаний для разделения объектов на круглые, эллиптические и гантелеобразные с использованием треугольной функции принадлежности, а для некруглых полосовых и ветвистых объектов - с использованием трапециевидной функции принадлежности на основе экспериментальных данных значений указанных коэффициентов округлости и заполнения и проводят распознавание доменов.
real: Разделения объектов; Коэффициент округлости;
model:  Разделение объектов на круглые, эллиптические и ветвистые;
---
Это приводит к локальному вертикальному движению границы раздела между текучей средой и связанным излучением энергии, что приводит к излучению волн сжатия в текучую среду
real: Излучение волн сжатия; Энергия в текучей среде;
model:  Излучение волн сжатия; Текучая среда;
---
Конвекция – это вид теплообмена, при котором внутренняя энергия передается струями и потоками.
real: Теплообмен; Внутренняя энергия;
model:  Тепловая энергия;
---
Когда число атомов в возбуждённом

Теперь посмотрим, насколько хорошо модель справляется с ответами на вопросы, которые она не видела.

In [ ]:
sample = df_test.sample(5)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', answer(row.X))
    print('---')

Внутреннее отражение — явление отражения электромагнитных или звуковых волн от границы раздела двух сред при условии, что волна падает из среды, где скорость её распространения меньше (в случае световых лучей это соответствует большему показателю преломления).
real: Луч света;
model:  Свет; Скорость распространения волн;
---
Гидравлические потери или гидравлическое сопротивление — безвозвратные потери удельной энергии (переход её в теплоту) на участках гидравлических систем (систем гидропривода, трубопроводах, другом гидрооборудовании), обусловленные наличием вязкого трения.
real: Теплота; Появление вязкого трения;
model:  Потери удельной энергии; Потери удельной энергии;
---
Свет, испускаемый светоизлучающими элементами, подвергается многократным отражениям в нижнем резонаторе осветительного модуля и, как результат, эффективно смешивается перед прохождением через апертуру в верхний резонатор осветительного модуля.
real: Свет; Многократное отражение;
model:  Свет;
---
При использовании

In [ ]:
# https://yandex.ru/patents/doc/RU216836U1_20230302
text = 'Полезная модель относится к области медицинской техники, в частности к массажным устройствам для ног. Релаксирующее устройство на основе нагревательных элементов содержит бокс, заполненный массирующим наполнителем в виде песка или песочной смеси, нагревательные элементы, устройство управления нагревательными элементами, панель управления нагревательными элементами, выполненная с возможностью выбора программы, времени и температуры работы нагревательных элементов. В корпусе бокса размещены нагревательные элементы, соединенные с устройством управления нагревательными элементами и панелью управления нагревательными элементами. Технический результат заключается в повышении эффективности воздействия на стопы ног в процессе проведения теплотерапии, улучшении процесса массажа. Достижение указанного технического результата обеспечивается за счёт управления процессом нагрева наполнителя в боксе с помощью нагревательных элементов. 1 з.п. ф-лы, 2 ил.'

for q in [text]:
    print(q)
    print()
    print(answer(q, do_sample=True, top_p=1))


Полезная модель относится к области медицинской техники, в частности к массажным устройствам для ног. Релаксирующее устройство на основе нагревательных элементов содержит бокс, заполненный массирующим наполнителем в виде песка или песочной смеси, нагревательные элементы, устройство управления нагревательными элементами, панель управления нагревательными элементами, выполненная с возможностью выбора программы, времени и температуры работы нагревательных элементов. В корпусе бокса размещены нагревательные элементы, соединенные с устройством управления нагревательными элементами и панелью управления нагревательными элементами. Технический результат заключается в повышении эффективности воздействия на стопы ног в процессе проведения теплотерапии, улучшении процесса массажа. Достижение указанного технического результата обеспечивается за счёт управления процессом нагрева наполнителя в боксе с помощью нагревательных элементов. 1 з.п. ф-лы, 2 ил.

Нагревательный наполнитель; Песочная смесь;


In [ ]:
new_model_name = 'modelT5_large_fiz'
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('modelT5_large_fiz/tokenizer_config.json',
 'modelT5_large_fiz/special_tokens_map.json',
 'modelT5_large_fiz/spiece.model',
 'modelT5_large_fiz/added_tokens.json')

In [ ]:
import shutil
from google.colab import files

folder_path = '/content/modelT5_large_fiz'

shutil.make_archive('/content/modelT5_large_fiz', 'zip', folder_path)

'/content/modelT5_large_fiz.zip'

In [ ]:
import shutil

source_path = '/content/modelT5_large_fiz.zip'
destinton_path = '/content/drive/My Drive/modelT5_large_fiz.zip'

shutil.copy(source_path, destinton_path)

'/content/drive/My Drive/modelT5_large_fiz.zip'

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Создаем список для хранения предсказаний и истинных значений
predictions = []
true_labels = []

# Генерируем предсказания на тестовом наборе данных
for example in tqdm(df_test[['X', 'Y']].values):
    # Предсказываем ответ модели
    predicted_answer = answer(example[0])
    predictions.append(predicted_answer)
    true_labels.append(example[1])

# Переводим предсказания и истинные метки в подходящий формат
# Можно использовать какую-либо обработку текста для более точного сравнения

# Пример простой обработки текста для предсказаний и истинных значений
predictions = [pred.strip().lower() for pred in predictions]
true_labels = [label.strip().lower() for label in true_labels]

  0%|          | 0/118 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# Вычисляем точность, полноту и F-меру
precision = precision_score(true_labels, predictions, average='binary', zero_division=0)
recall = recall_score(true_labels, predictions, average='binary', zero_division=0)
f1 = f1_score(true_labels, predictions, average='binary', zero_division=0)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from tqdm.auto import tqdm

def compute_metrics(pred_labels, true_labels):
    precision = precision_score(true_labels, pred_labels, average='micro', zero_division=0)
    recall = recall_score(true_labels, pred_labels, average='micro', zero_division=0)
    f1 = f1_score(true_labels, pred_labels, average='micro', zero_division=0)
    return precision, recall, f1

def evaluate_model(model, tokenizer, test_data):
    model.eval()
    predictions = []
    true_labels = []
    for example in tqdm(test_data):
        inputs = tokenizer(example[0], return_tensors='pt').to(model.device)
        with torch.no_grad():
            hypothesis = model.generate(**inputs, num_beams=5, max_new_tokens=50)  # Например, максимум 50 новых токенов
        predicted_label = tokenizer.decode(hypothesis[0], skip_special_tokens=True)
        predictions.append(predicted_label)
        true_labels.append(example[1])
    return compute_metrics(predictions, true_labels)

# Далее в коде обучения после завершения обучения вызовите функцию оценки:
precision, recall, f1 = evaluate_model(model, tokenizer, df_test[['X', 'Y']].values.tolist())
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1}')


  0%|          | 0/118 [00:00<?, ?it/s]

Precision: 0.05084745762711865
Recall: 0.05084745762711865
F1 Score: 0.050847457627118654
